In [2]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd

import scanpy as sc
import scipy as sp

import sys
import os
sys.path.append(os.path.abspath("./utility_functions"))

import rz_functions as rz
import ng_functions as ng
import rz_utility_spring as srz

from time import time
#from seaborn import kdeplot, distplot, pairplot
#from seaborn import Pairgrid

/home/ng136/miniconda3/envs/py36/lib/python3.6/site-packages/anndata/_core/anndata.py:21: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import RangeIndex


python version: 3.6.10


In [3]:
cell_subset_dict = {'B cells' : 'B cells',
                     'Basophils' : 'Basophils',
                     'DC1' : 'MoMacDC',
                     'DC2' : 'MoMacDC',
                     'DC3' : 'MoMacDC',
                     'Mac1' : 'MoMacDC',
                     'Mac2' : 'MoMacDC',
                     'Mac3' : 'MoMacDC',
                     'Mac4' : 'MoMacDC',
                     'Mono1' : 'MoMacDC',
                     'Mono2' : 'MoMacDC',
                     'Mono3' : 'MoMacDC',
                     'MonoDC' : 'MoMacDC',
                     'N1' : 'Neutrophils',
                     'N2' : 'Neutrophils',
                     'N3' : 'Neutrophils',
                     'N4' : 'Neutrophils',
                     'N5' : 'Neutrophils',
                     'N6' : 'Neutrophils',
                     'N6' : 'Neutrophils',
                     'NK cells' : 'NK cells',
                     'T1' : 'T cells',
                     'T2' : 'T cells',
                     'T3' : 'T cells',
                     'pDC' : 'pDC'}


# Data loading, preprocessing and normalization

In [6]:
adata = sc.read_h5ad('backups/tox_marie_all_cells_unnormalized_pca_umap_leiden_35358x22631_backup_200412_21h30.h5ad')

adata[:,['Il12b']].X = adata[:,['Il12b', 'eYFP']].X.sum(1)
adata[:,['eYFP']].X = 0


sc.pp.normalize_total(adata, target_sum=1e4)

adata.obs['n_counts'] = adata.X.sum(1).A1
adata.obs.head()

,barcode,group,library_name,marker,max_mt,min_tot,mito_frac,n_counts,n_genes,pool,tissue,treatment,leiden
48-pool1,AAACAAACAGAGGATAAGACACCA,B,Liver_CD45pos_NT_B,CD45pos,0.15,200,0.040964,10000.000977,348,pool1,Liver,NT,8
98-pool1,AAACAAACAGAGGATACAATACTT,B,Liver_CD45pos_NT_B,CD45pos,0.15,200,0.005613,10000.000000,581,pool1,Liver,NT,20
237-pool1,AAACAAACAGAGGATAGGCCCTTA,B,Liver_CD45pos_NT_B,CD45pos,0.15,200,0.035451,10000.001953,470,pool1,Liver,NT,11
246-pool1,AAACAAACAGAGGATAGGGAGGTA,B,Liver_CD45pos_NT_B,CD45pos,0.15,200,0.018000,10000.000000,374,pool1,Liver,NT,16
342-pool1,AAACAAACAGAGGATATGGCTACC,B,Liver_CD45pos_NT_B,CD45pos,0.15,200,0.035088,10000.000000,318,pool1,Liver,NT,6


In [7]:
#loglikelihoods = sp.sparse.load_npz('backups/loglikelihoods_bay_classif_tox_marie_all_cells_200407_11h02.npz')
logls = np.load('backups/loglikelihoods_bay_classif_tox_marie_all_cells_200407_11h02.npz', allow_pickle=True)

mostlikely_subset = pd.DataFrame(logls['data'], index=logls['index'], columns=logls['columns']).idxmax()
mostlikely_subset = pd.Series(mostlikely_subset, dtype='category')
mostlikely_subset.index = adata.obs.index

mostlikely_celltype = mostlikely_subset.map(cell_subset_dict).astype('category')

In [8]:
adata.obs['cell_type'] = mostlikely_celltype
adata.obs['minor_subset'] = mostlikely_subset